In [24]:
import pandas as pd
import numpy as np
import openpyxl
from datetime import datetime

In [25]:
files = ['WP020204 - 20-Feb-2025 12-22-34-000 PM - MainEnergy.xls', 'WP020204 - 20-Feb-2025 12-22-34-000 PM - TODEnergy.xls']

In [26]:
serial_no = files[0].split("-")[0].strip()
serial_no

'WP020204'

In [27]:
# Read the Excel file
df = pd.read_excel(files[0])
df = df.drop(index=[0, 1]).reset_index(drop=True) #drop the first two rows

# assign values of the third row as column names
df.columns = df.iloc[0]
df = df.drop(0).reset_index(drop=True)
#df.head()

In [28]:
column_list = {"Billing Date":'billing_period',"Active(I) Total":"cumulative_import",
    "Active(E) Total":"cumulative_export","Apparent-Active(I) - type 2":"apparent_import",
    "Reactive(I)":"reactive_import","Reactive(E)":"reactive_export"}

df = df.rename(columns=column_list)
df = df[df["History"] != 0] #drop this row because we can not parse its date string (its empty)
df['billing_period'] = pd.to_datetime(df['billing_period']).dt.normalize()
df['month'] = pd.to_datetime(df['billing_period']).dt.month_name()
df['year'] = pd.to_datetime(df['billing_period']).dt.year

# select needed columns
column_list = ['History', 'billing_period', 'cumulative_import', 'cumulative_export',
      'apparent_import', 'reactive_import', 'reactive_export','month', 'year']
df = df[column_list]
#df = df[df["History"] == 1]
df.head()

,History,billing_period,cumulative_import,cumulative_export,apparent_import,reactive_import,reactive_export,month,year
1,1,2025-02-01,176406172.578899,45036.513,187551818.269299,51925429.9622,899196.0148,February,2025
2,2,2025-01-01,170227521.205,43139.556,180877936.302899,49919915.6973,883925.8009,January,2025
3,3,2024-12-01,164393093.199699,42768.7731,174709372.8919,48128523.3858,882430.1424,December,2024
4,4,2024-11-01,158545581.6502,42182.8183,168517109.958799,46323894.1451,878456.6002,November,2024
5,5,2024-10-01,152804300.599,42182.8055,162411024.3161,44461302.9224,874835.434,October,2024


### GETTING TOU VALUES

In [29]:
df2 = pd.read_excel(files[1])
df2 = df2.drop(index=[0, 1]).reset_index(drop=True) #drop the first two rows

# assign values of the third row as column names
df2.columns = df2.iloc[0]
df2 = df2.drop(0).reset_index(drop=True)
df2_columns = ['History', 'Billing Date', 'TOD/SLAB', 'Active(I) Total','Active(E) Total']
df2 = df2[df2_columns]
df2.head(3)

,History,Billing Date,TOD/SLAB,Active(I) Total,Active(E) Total
0,0,----,1,96256715.1213,25016.595
1,0,----,2,58640935.1716,1532.7879
2,0,----,3,25970470.2811,18548.5367


In [30]:
df2_columns = {'Billing Date':'billing_period', 'TOD/SLAB':'tou', 'Active(I) Total':'import',
               'Active(E) Total':'export'}
df2 = df2.rename(columns=df2_columns)
df2 = df2[df2["History"] != 0] #drop the current values. also, we cannot parse its date(empty)
df['billing_period'] = pd.to_datetime(df['billing_period']).dt.normalize()
df['month'] = pd.to_datetime(df['billing_period']).dt.month_name()
df['year'] = pd.to_datetime(df['billing_period']).dt.year
#df2 = df2[df2["History"] == 1]
df2.head()

,History,billing_period,tou,import,export
3,1,2025-02-01 00:00:00,1,93887499.2142,24955.1705
4,1,2025-02-01 00:00:00,2,57194922.7297,1532.7879
5,1,2025-02-01 00:00:00,3,25204181.6104,18548.5367
6,2,2025-01-01 00:00:00,1,90692175.6319,24859.7326
7,2,2025-01-01 00:00:00,2,55184111.7331,1532.7879


In [31]:
history_list = df['History'].tolist()

In [33]:
rates1, rates2, rates3, rates4, rates5, rates6 = [],[],[],[],[],[]
for history in history_list:
    # getting rates and and assiging them to df
    if df['History'].equals(df2['History']) and df['billing_period'].equals(df2['billing_period']):
        df['rate1'] = df2[(df2['tou'] == 1) & (df2['History'] == history)]['import'].values[0]
        df['rate2'] = df2[(df2['tou'] == 2) & (df2['History'] == history)]['import'].values[0]
        df['rate3'] = df2[(df2['tou'] == 3) & (df2['History'] == history)]['import'].values[0]

        df['rate4'] = df2[(df2['tou'] == 1) & (df2['History'] == history)]['export'].values[0]
        df['rate5'] = df2[(df2['tou'] == 2) & (df2['History'] == history)]['export'].values[0]
        df['rate6'] = df2[(df2['tou'] == 3) & (df2['History'] == history)]['export'].values[0]

df

,History,billing_period,cumulative_import,cumulative_export,apparent_import,reactive_import,reactive_export,month,year
1,1,2025-02-01,176406172.578899,45036.513,187551818.269299,51925429.9622,899196.0148,February,2025
2,2,2025-01-01,170227521.205,43139.556,180877936.302899,49919915.6973,883925.8009,January,2025
3,3,2024-12-01,164393093.199699,42768.7731,174709372.8919,48128523.3858,882430.1424,December,2024
4,4,2024-11-01,158545581.6502,42182.8183,168517109.958799,46323894.1451,878456.6002,November,2024
5,5,2024-10-01,152804300.599,42182.8055,162411024.3161,44461302.9224,874835.434,October,2024
6,6,2024-09-01,146985425.6646,42168.4945,156193793.7461,42540535.0956,868022.7941,September,2024
7,7,2024-08-01,140967484.871299,35549.9017,149830925.7882,40759525.8709,862382.4484,August,2024
8,8,2024-07-01,134835499.7527,35549.8992,143413429.5976,39285588.1761,786495.2135,July,2024
9,9,2024-06-01,127899872.042899,35549.8692,136387477.194299,38604269.7574,591937.5956,June,2024
10,10,2024-05-01,120982242.390099,35549.8672,129367589.8303,37712654.6068,529072.4632,May,2024


generate a dictionary 
dict_rates = {1:{'rate1':8494,"rate2":0,"rate3":90}}

In [34]:
df_history_is_1 = (df['History'] == 1).all()
df2_history_is_1 = (df2['History'] == 1).all()

df_history_is_1, df2_history_is_1

(np.False_, np.False_)

In [ ]:
df['History'].equals(df2['History'])

In [ ]:
# df['rate1'] = df['rate1'].apply(lambda x: '{:.2f}'.format(x))
# df['rate1']

1     50829326.00
2     50829326.00
3     50829326.00
4     50829326.00
5     50829326.00
6     50829326.00
7     50829326.00
8     50829326.00
9     50829326.00
10    50829326.00
11    50829326.00
12    50829326.00
13    50829326.00
14    50829326.00
15    50829326.00
Name: rate1, dtype: object

In [ ]:
# values_list =[]
# for history in history_list:
#     values = df2[(df2['tou'] == 1) & (df2['History'] == history)]['import'].values[0]
#     values_list.append(values)
# print(values_list)

[93887499.2141999]
[93887499.2141999, 90692175.6318999]
[93887499.2141999, 90692175.6318999, 87656603.2758]
[93887499.2141999, 90692175.6318999, 87656603.2758, 84571395.1511]
[93887499.2141999, 90692175.6318999, 87656603.2758, 84571395.1511, 81488614.6323999]
[93887499.2141999, 90692175.6318999, 87656603.2758, 84571395.1511, 81488614.6323999, 78304837.9553]
[93887499.2141999, 90692175.6318999, 87656603.2758, 84571395.1511, 81488614.6323999, 78304837.9553, 75178756.8141]
[93887499.2141999, 90692175.6318999, 87656603.2758, 84571395.1511, 81488614.6323999, 78304837.9553, 75178756.8141, 72000036.1759999]
[93887499.2141999, 90692175.6318999, 87656603.2758, 84571395.1511, 81488614.6323999, 78304837.9553, 75178756.8141, 72000036.1759999, 68461186.0465999]
[93887499.2141999, 90692175.6318999, 87656603.2758, 84571395.1511, 81488614.6323999, 78304837.9553, 75178756.8141, 72000036.1759999, 68461186.0465999, 64881196.6081999]
[93887499.2141999, 90692175.6318999, 87656603.2758, 84571395.1511, 81488

In [ ]:
df['serial_no'] = serial_no 
df['reading_date'] = datetime.now().replace(second=0, microsecond=0)
df['vt_ratio'] = pd.NA  
df['ct_ratio'] = pd.NA  
df['no_of_resets'] = pd.NA  
df['date_of_last_reset'] = pd.NA  
df['programing_count'] = pd.NA 
df['vt_ratio'] = pd.NA  
df['ct_ratio'] = pd.NA  
df['no_of_resets'] = pd.NA  
df['date_of_last_reset'] = pd.NA  
df['programing_count'] = pd.NA 
 

In [ ]:
columns_to_assign_na = ['vt_ratio', 'ct_ratio', 'no_of_resets', 'date_of_last_reset', 'programing_count',
                        'max_dem1','max_dem1_datetime','max_dem2','max_dem2_datetime','max_dem3','max_dem3_datetime']

for column in columns_to_assign_na:
    df[column] = pd.NA

df

,History,billing_period,cumulative_import,cumulative_export,apparent_import,reactive_import,reactive_export,month,year,rate1,...,ct_ratio,no_of_resets,date_of_last_reset,programing_count,max_dem1,max_dem1_datetime,max_dem2,max_dem2_datetime,max_dem3,max_dem3_datetime
1,1,2025-02-01,176406172.578899,45036.513,187551818.269299,51925429.9622,899196.0148,February,2025,5.082933e+07,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2,2,2025-01-01,170227521.205,43139.556,180877936.302899,49919915.6973,883925.8009,January,2025,5.082933e+07,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3,3,2024-12-01,164393093.199699,42768.7731,174709372.8919,48128523.3858,882430.1424,December,2024,5.082933e+07,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
4,4,2024-11-01,158545581.6502,42182.8183,168517109.958799,46323894.1451,878456.6002,November,2024,5.082933e+07,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
5,5,2024-10-01,152804300.599,42182.8055,162411024.3161,44461302.9224,874835.434,October,2024,5.082933e+07,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
6,6,2024-09-01,146985425.6646,42168.4945,156193793.7461,42540535.0956,868022.7941,September,2024,5.082933e+07,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
7,7,2024-08-01,140967484.871299,35549.9017,149830925.7882,40759525.8709,862382.4484,August,2024,5.082933e+07,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
8,8,2024-07-01,134835499.7527,35549.8992,143413429.5976,39285588.1761,786495.2135,July,2024,5.082933e+07,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
9,9,2024-06-01,127899872.042899,35549.8692,136387477.194299,38604269.7574,591937.5956,June,2024,5.082933e+07,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
10,10,2024-05-01,120982242.390099,35549.8672,129367589.8303,37712654.6068,529072.4632,May,2024,5.082933e+07,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


### FOR INSERTING MULTIPLE MONTHS